# Create .PNG images of all timesteps in IDX firesmoke dataset

## Import necessary libraries

In [3]:
# for numerical work
import numpy as np

# for accessing file system
import os

# for downloading latest firesmoke netCDF
import requests

# for loading netcdf files, for metadata
import xarray as xr
# for connecting OpenVisus framework to xarray
# from https://github.com/sci-visus/openvisuspy, 
from openvisuspy.xarray_backend import OpenVisusBackendEntrypoint

# Used for processing netCDF time data
import time
import datetime

# Used for indexing via metadata
import pandas as pd

# for plotting
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# for exporting the dictionary of issue files at the end of notebook
import pickle

# Stores the OpenVisus cache in the local direcrtory 
import os
os.environ["VISUS_CACHE"]="./visus_cache_can_be_erased"

# Accessory, used to generate progress bar for running for loops
# from tqdm.notebook import tqdm
# import ipywidgets
# import jupyterlab_widgets
from tqdm import tqdm

from OpenVisus import *

### In this section, we load our data using `xr.open_dataset`.

In [4]:
# path to tiny NetCDF
url = 'https://github.com/sci-visus/NSDF-WIRED/raw/main/data/firesmoke_metadata_recent.nc'

# Download the file using requests
response = requests.get(url)
local_netcdf = 'firesmoke_metadata.nc'
with open(local_netcdf, 'wb') as f:
    f.write(response.content)
    
# open tiny netcdf with xarray and OpenVisus backend
ds = xr.open_dataset(local_netcdf, engine=OpenVisusBackendEntrypoint)

ov.LoadDataset(http://atlantis.sci.utah.edu/mod_visus?dataset=UBC_fire_smoke_BSC&cached=1)
PM25
Adding field  PM25 shape  [25053, 381, 1081, 21] dtype  float32 labels  ['time', 'ROW', 'COL', 'resolution'] Max Resolution  20


## Calculate derived metadata using original metadata above to create coordinates
### This is required to allow for indexing of data via metadata

#### Calculate latitude and longitude grid

In [5]:
# Get metadata to compute lon and lat
xorig = ds.XORIG
yorig = ds.YORIG
xcell = ds.XCELL
ycell = ds.YCELL
ncols = ds.NCOLS
nrows = ds.NROWS

longitude = np.linspace(xorig, xorig + xcell * (ncols - 1), ncols)
latitude = np.linspace(yorig, yorig + ycell * (nrows - 1), nrows)

#### Using calculated latitude and longitude, create coordinates allowing for indexing data using lat/lon

In [6]:
# Create coordinates for lat and lon (credit: Aashish Panta)
ds.coords['lat'] = ('ROW', latitude)
ds.coords['lon'] = ('COL', longitude)

# Replace col and row dimensions with newly calculated lon and lat arrays (credit: Aashish Panta)
ds = ds.swap_dims({'COL': 'lon', 'ROW': 'lat'})

## Get timestamps to label video frames
Need to use `idx_calls` generated during conversion

In [ ]:
# Load idx_calls from file
with open('idx_calls_v4.pkl', 'rb') as f:
    idx_calls = pickle.load(f)

print(len(idx_calls))

##### Return an array of the tflags as pandas timestamps

In [ ]:
# to store pandas timestamps
timestamps = []

# convert all tflags to pandas timestamps, store in timestamps list
for call in idx_calls:
    timestamps.append(pd.Timestamp(call[2]))

# check out the first 3 timestamps
print(timestamps[0:3])

## Create the video
### note: ffmpeg needs to be installed for this

In [ ]:
# set parameters for indexing data
data_resolution = 0
folder = "/usr/sci/scratch_nvme/arleth/dump/idx_frames"

# set parameters for creating visualization of each timestep with matplotlib
my_norm = "log"
my_extent = [np.min(longitude), np.max(longitude), np.min(latitude), np.max(latitude)]
my_aspect = 'auto'
my_origin = 'lower'
my_cmap = 'hot'

# to keep track of files that fail to visualized into .PNG
issue_files = {}

# for all timesteps create visualization of firesmoke at time
for i in tqdm(range(len(idx_calls))):
    # get PM25 values and provide 4 values, the colons mean select all lat and lon indices
    data_array_at_time = ds['PM25'].loc[i, :, :, data_resolution]

    # create visualization using matplotlib and cartopy geography lines, 
    # catch exceptions accordingly
    try:
        my_fig, my_plt = plt.subplots(figsize=(15, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))
        plot = my_plt.imshow(data, norm=my_norm, extent=my_extent, aspect=my_aspect, origin=my_origin, cmap=my_cmap)
        my_fig.colorbar(plot,location='right', label='ug/m^3')
        my_plt.coastlines()
        my_plt.gridlines(draw_labels=True)
        my_fig.suptitle(f'Ground level concentration of PM2.5 microns and smaller {timestamps[i]}\n')
        # add caption showing this is from IDX dataset
        my_plt.text(0.5, -0.1, 'IDX Data', ha='center', va='center', transform=my_plt.transAxes)
        
        # # save visualization as a .PNG to our folder
        plt.savefig(folder + "/frames%05d.png" % i, dpi=280)
        plt.close(my_fig);  # close the figure after saving
        # plt.show()
        matplotlib.pyplot.close()
    except:
        issue_files[timestamps[i]] = data
        continue

In [ ]:
# save 'issue_files' to review
with open('new_idx_issues.pkl', 'wb') as f:
    pickle.dump(issue_files, f)

In [ ]:
with open('new_idx_issues.pkl', 'rb') as f:
    new_idx_issues = pickle.load(f)
print(f'len of new_idx_issues.pkl = {len(new_idx_issues)}')